In [1]:
import pandas as pd
import random

In [2]:
df_vehicles = pd.read_excel('../Datos_P1/df_vehicle.xlsx')
df_distances_km = pd.read_excel('../Datos_P1/df_distance_km.xlsx')
df_orders = pd.read_excel('../Datos_P1/df_orders.xlsx')
df_location = pd.read_excel('../Datos_P1/df_location.xlsx')

df_distances_km.index = df_distances_km.columns

In [3]:
def create_initial_population(population_size=10):
    population = []
    
    orders = df_orders.to_dict(orient="records")
    vehicles = df_vehicles.to_dict(orient="records")
    almacen = "Almacén"
    
    for _ in range(population_size):
        individual = []
        remaining_orders = orders.copy()  # Clonar los pedidos restantes
        
        for vehicle in vehicles:
            route = [almacen]  # Cada ruta comienza en el almacén
            capacity_left = vehicle["capacidad_kg"]
            autonomy_left = vehicle["autonomia_km"]
            
            while remaining_orders:
                order = random.choice(remaining_orders)

                if order["order_demand"] <= capacity_left:
                    current_client = route[len(route) - 1]
                    next_client = order["cliente"]
                    distance = df_distances_km.at[current_client, next_client]

                    if distance <= autonomy_left and distance != 0:
                        route.append(next_client)
                        capacity_left -= order["order_demand"]
                        autonomy_left -= distance
                        remaining_orders.remove(order)
                    else:
                        break
                else:
                    break
            
            route.append(almacen)
            individual.append(route)
        
        population.append(individual)
    
    return population

In [4]:
def evaluate_population(population):
    fitness_scores = []
    
    # Crear un diccionario de los pedidos por cliente para una búsqueda más eficiente
    orders_dict = df_orders.set_index('cliente')['order_demand'].to_dict()

    for individual in population:
        total_distance = 0
        total_penalty = 0  # Penalización si hay algún error (capacidad, autonomía)
        
        for route, vehicle in zip(individual, df_vehicles.to_dict(orient="records")):
            route_distance = 0
            capacity_left = vehicle["capacidad_kg"]
            autonomy_left = vehicle["autonomia_km"]
            
            # Verifica la distancia y la capacidad/autonomía en cada ruta
            current_location = "Almacén"
            
            for client in route[1:-1]:  # Saltar el almacén al inicio y final
                distance = df_distances_km.at[current_location, client]
                
                # Asegurarse de que el cliente esté en la matriz de distancias
                if client not in df_distances_km.index or current_location not in df_distances_km.columns:
                    print(f"Warning: {current_location} or {client} not found in the distance matrix.")
                    continue
                
                route_distance += distance
                autonomy_left -= distance  # Reducir autonomía con la distancia recorrida
                current_location = client
                
                # Obtener la demanda del pedido del cliente actual de la lista de pedidos
                order_demand = orders_dict.get(client, 0)  # Si no hay pedido, asignamos 0

                # Verificar si la autonomía o capacidad se exceden
                if autonomy_left < 0:
                    total_penalty += 1000  # Penalización por exceder la autonomía
                
                capacity_left -= order_demand  # Reducir la capacidad disponible
                if capacity_left < 0:
                    total_penalty += 1000  # Penalización por exceder la capacidad

            # Añadir la distancia de regreso al almacén
            route_distance += df_distances_km.at[current_location, "Almacén"]
            total_distance += route_distance
        
        # Al final de la ruta, guardamos la aptitud de esta solución
        fitness_scores.append(total_distance + total_penalty)
    
    return fitness_scores

In [5]:
def seleccion_mejor_fitness(population, fitness_scores, num_seleccionados=2):
    # Combina la población con sus puntuaciones de fitness
    data = list(zip(population, fitness_scores))
    
    # Ordena la población por la puntuación de fitness en orden descendente
    data_sorted = sorted(data, key=lambda x: x[1], reverse=True)
    
    # Extrae los 'num_seleccionados' mejores individuos
    seleccionados = [individuo for individuo, _ in data_sorted[:num_seleccionados]]
    
    return seleccionados

In [6]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))  # Determinar el rango de intercambio

    # Crear la descendencia con valores vacíos
    offspring = [None] * size

    # Copiar un segmento de parent1 al hijo
    offspring[start:end] = parent1[start:end]

    # Llenar las posiciones restantes con los elementos de parent2, respetando el orden
    current_index = 0
    for i in range(size):
        if offspring[i] is None:
            while parent2[current_index] in offspring:
                current_index += 1
            offspring[i] = parent2[current_index]
    
    return offspring

In [7]:
def mutation_swap(offspring):
    size = len(offspring)
    idx1, idx2 = random.sample(range(1, size-1), 2)  # Evitar intercambiar 'Almacén'
    offspring[idx1], offspring[idx2] = offspring[idx2], offspring[idx1]
    return offspring

In [8]:
def elitismo(population, fitness_scores, offspring, num_elites=1):
    # Combinamos la población actual con la descendencia
    combined_population = population + [offspring]
    
    # Evaluamos la descendencia por separado (porque offspring no está evaluado aún)
    fitness_offspring = evaluate_population([offspring])[0]  # Evaluar solo la descendencia
    combined_fitness = fitness_scores + [fitness_offspring]
    
    # Ordenamos los individuos por fitness (en orden descendente)
    sorted_population = [indiv for _, indiv in sorted(zip(combined_fitness, combined_population), reverse=True)]
    
    # Seleccionamos los mejores 'num_elites' individuos
    new_population = sorted_population[:len(population) - num_elites]  # Mantener los mejores
    new_population += sorted_population[:num_elites]  # Añadir los mejores élites
    
    return new_population

In [20]:
def genetic_algorithm(mutation_probability=0.9, num_elites=1, max_generations=4, fitness_threshold=None):
    population = create_initial_population()  # Crear la población inicial
    fitness_scores = evaluate_population(population=population)  # Evaluar la aptitud de la población
    
    best_fitness = min(fitness_scores)
    generation = 0
    
    while generation < max_generations:
        parents = seleccion_mejor_fitness(population=population, fitness_scores=fitness_scores)
        offspring = crossover_order(parents[0], parents[1])

        # Mutación
        if random.random() < mutation_probability:
            offspring = mutation_swap(offspring)
        
        # Reemplazo con elitismo
        population = elitismo(population, fitness_scores, offspring, num_elites=num_elites)
        
        # Re-calcular las puntuaciones de fitness
        fitness_scores = evaluate_population(population=population)
        
        # Revisar si la mejora es menor que un umbral
        if fitness_threshold and abs(best_fitness - min(fitness_scores)) < fitness_threshold:
            break  # Detener si no hay mejora significativa
        
        best_fitness = min(fitness_scores)
        generation += 1

    return population

In [21]:
full_population = genetic_algorithm()

In [22]:
for individual in full_population:
    print("New Population")
    for route in individual:
        print(route)

New Population
['Almacén', 'Cliente_4', 'Cliente_12', 'Almacén']
['Almacén', 'Cliente_4', 'Cliente_3', 'Cliente_2', 'Cliente_9', 'Almacén']
['Almacén', 'Cliente_18', 'Cliente_13', 'Almacén']
['Almacén', 'Cliente_11', 'Cliente_15', 'Cliente_20', 'Cliente_5', 'Cliente_1', 'Almacén']
['Almacén', 'Cliente_10', 'Cliente_5', 'Almacén']
['Almacén', 'Cliente_7', 'Cliente_19', 'Cliente_6', 'Almacén']
New Population
['Almacén', 'Cliente_4', 'Cliente_12', 'Almacén']
['Almacén', 'Cliente_11', 'Cliente_15', 'Cliente_20', 'Cliente_5', 'Cliente_1', 'Almacén']
['Almacén', 'Cliente_18', 'Cliente_13', 'Almacén']
['Almacén', 'Cliente_4', 'Cliente_3', 'Cliente_2', 'Cliente_9', 'Almacén']
['Almacén', 'Cliente_10', 'Cliente_5', 'Almacén']
['Almacén', 'Cliente_7', 'Cliente_19', 'Cliente_6', 'Almacén']
New Population
['Almacén', 'Cliente_4', 'Cliente_12', 'Almacén']
['Almacén', 'Cliente_11', 'Cliente_15', 'Cliente_20', 'Cliente_5', 'Cliente_1', 'Almacén']
['Almacén', 'Cliente_18', 'Cliente_13', 'Almacén']
['A